# FrameWright - Video Restoration (Google Colab)

**Free GPU-accelerated video restoration in the cloud!**

This notebook lets you restore old/degraded videos using AI upscaling with a free GPU.

## Features
- 4x AI upscaling (Real-ESRGAN)
- Frame deduplication (for old films)
- Frame interpolation (RIFE)
- YouTube URL support

## Instructions
1. **Runtime > Change runtime type > GPU (T4)**
2. Run each cell in order
3. Download your restored video at the end

---

## Step 1: Check GPU & Install Dependencies

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("WARNING: No GPU detected! Go to Runtime > Change runtime type > GPU")

In [ ]:
# Install dependencies
!pip install -q yt-dlp pillow numpy opencv-python-headless imagehash

# Install Real-ESRGAN ncnn-vulkan (pre-compiled for Linux)
!mkdir -p ~/.framewright/bin
!wget -q https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesrgan-ncnn-vulkan-20220424-ubuntu.zip -O /tmp/realesrgan.zip
!unzip -q -o /tmp/realesrgan.zip -d ~/.framewright/bin/
!chmod +x ~/.framewright/bin/realesrgan-ncnn-vulkan

# Install RIFE for frame interpolation
!wget -q https://github.com/nihui/rife-ncnn-vulkan/releases/download/20221029/rife-ncnn-vulkan-20221029-ubuntu.zip -O /tmp/rife.zip
!unzip -q -o /tmp/rife.zip -d ~/.framewright/bin/
!chmod +x ~/.framewright/bin/rife-ncnn-vulkan

# Install FFmpeg
!apt-get -qq install ffmpeg

print("\n✅ Dependencies installed!")

## Step 2: Configure Your Video

In [ ]:
#@title Video Source Configuration
#@markdown ### Choose ONE source:

#@markdown **Option A: YouTube URL**
youtube_url = "" #@param {type:"string"}

#@markdown **Option B: Upload a file** (run this cell, then use the file picker below)
upload_file = False #@param {type:"boolean"}

#@markdown ---
#@markdown ### Settings
scale_factor = 4 #@param [2, 4] {type:"integer"}
output_format = "mp4" #@param ["mp4", "mkv", "webm"]
quality_crf = 18 #@param {type:"slider", min:10, max:28, step:1}

#@markdown ### Frame Deduplication (for old films)
enable_deduplication = True #@param {type:"boolean"}
dedup_threshold = 0.98 #@param {type:"slider", min:0.90, max:0.99, step:0.01}

#@markdown ### Frame Interpolation (RIFE)
enable_interpolation = True #@param {type:"boolean"}
target_fps = 24 #@param [18, 24, 30, 48, 60] {type:"integer"}

# Handle file upload
video_path = None
if upload_file:
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        video_path = list(uploaded.keys())[0]
        print(f"\n✅ Uploaded: {video_path}")
elif youtube_url:
    print(f"\n✅ Will download from YouTube: {youtube_url}")
else:
    print("\n⚠️ Please provide a YouTube URL or enable file upload!")

## Step 3: Run Restoration

In [ ]:
import os
import subprocess
import time
from pathlib import Path
import json

# Setup directories
WORK_DIR = Path("/content/framewright_output")
WORK_DIR.mkdir(exist_ok=True)
FRAMES_DIR = WORK_DIR / "frames"
ENHANCED_DIR = WORK_DIR / "enhanced"
FRAMES_DIR.mkdir(exist_ok=True)
ENHANCED_DIR.mkdir(exist_ok=True)

REALESRGAN = Path.home() / ".framewright/bin/realesrgan-ncnn-vulkan"
RIFE = Path.home() / ".framewright/bin/rife-ncnn-vulkan"

# Model mapping
MODEL_MAP = {
    2: "realesrgan-x2plus",
    4: "realesrgan-x4plus"
}

def download_youtube(url, output_dir):
    """Download video from YouTube."""
    print(f"📥 Downloading from YouTube...")
    output_path = output_dir / "source.mp4"
    cmd = [
        "yt-dlp",
        "-f", "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best",
        "-o", str(output_path),
        url
    ]
    subprocess.run(cmd, check=True)
    return output_path

def get_video_info(video_path):
    """Get video metadata using ffprobe."""
    cmd = [
        "ffprobe", "-v", "quiet", "-print_format", "json",
        "-show_format", "-show_streams", str(video_path)
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return json.loads(result.stdout)

def extract_frames(video_path, frames_dir):
    """Extract frames from video."""
    print(f"🎞️ Extracting frames...")
    cmd = [
        "ffmpeg", "-i", str(video_path),
        "-qscale:v", "1", "-qmin", "1", "-qmax", "1",
        str(frames_dir / "frame_%08d.png")
    ]
    subprocess.run(cmd, check=True, capture_output=True)
    frames = sorted(frames_dir.glob("frame_*.png"))
    print(f"   Extracted {len(frames)} frames")
    return frames

def deduplicate_frames(frames, threshold=0.98):
    """Remove duplicate frames using perceptual hashing."""
    print(f"🔍 Detecting duplicate frames (threshold={threshold})...")
    try:
        import imagehash
        from PIL import Image
    except ImportError:
        print("   imagehash not available, skipping deduplication")
        return frames

    unique_frames = []
    last_hash = None

    for frame in frames:
        img = Image.open(frame)
        current_hash = imagehash.dhash(img, hash_size=16)

        if last_hash is None:
            unique_frames.append(frame)
            last_hash = current_hash
        else:
            # Calculate similarity
            max_dist = 16 * 16
            distance = current_hash - last_hash
            similarity = 1.0 - (distance / max_dist)

            if similarity < threshold:
                unique_frames.append(frame)
                last_hash = current_hash

    pct = len(unique_frames) / len(frames) * 100
    print(f"   {len(unique_frames)}/{len(frames)} unique frames ({pct:.1f}%)")
    return unique_frames

def enhance_frames(frames, output_dir, scale=4, model="realesrgan-x4plus"):
    """Enhance frames using Real-ESRGAN."""
    print(f"🚀 Enhancing {len(frames)} frames with Real-ESRGAN ({scale}x)...")
    total = len(frames)
    start_time = time.time()

    for i, frame in enumerate(frames):
        output_path = output_dir / frame.name
        if output_path.exists():
            continue

        cmd = [
            str(REALESRGAN),
            "-i", str(frame),
            "-o", str(output_path),
            "-n", model,
            "-s", str(scale),
            "-f", "png"
        ]
        subprocess.run(cmd, capture_output=True)

        # Progress
        if (i + 1) % 10 == 0 or i == total - 1:
            elapsed = time.time() - start_time
            fps = (i + 1) / elapsed
            eta = (total - i - 1) / fps if fps > 0 else 0
            print(f"   {i+1}/{total} ({fps:.1f} fps, ETA: {eta/60:.1f} min)")

    print(f"   ✅ Enhancement complete!")
    return sorted(output_dir.glob("frame_*.png"))

def reassemble_video(frames_dir, output_path, fps, crf=18, audio_path=None):
    """Reassemble frames into video."""
    print(f"🎬 Reassembling video at {fps} fps...")
    cmd = [
        "ffmpeg", "-y",
        "-framerate", str(fps),
        "-i", str(frames_dir / "frame_%08d.png"),
    ]

    if audio_path and audio_path.exists():
        cmd.extend(["-i", str(audio_path), "-c:a", "aac", "-b:a", "192k"])

    cmd.extend([
        "-c:v", "libx264",
        "-crf", str(crf),
        "-pix_fmt", "yuv420p",
        str(output_path)
    ])

    subprocess.run(cmd, check=True, capture_output=True)
    print(f"   ✅ Saved to: {output_path}")
    return output_path

# Main restoration pipeline
print("="*60)
print("  FRAMEWRIGHT VIDEO RESTORATION (Colab)")
print("="*60)

# Get source video
if youtube_url:
    source_video = download_youtube(youtube_url, WORK_DIR)
elif video_path:
    source_video = Path(video_path)
else:
    raise ValueError("No video source provided!")

# Get video info
info = get_video_info(source_video)
video_stream = next((s for s in info['streams'] if s['codec_type'] == 'video'), None)
if video_stream:
    fps_str = video_stream.get('r_frame_rate', '25/1')
    num, den = map(int, fps_str.split('/'))
    source_fps = num / den if den else 25
    width = video_stream.get('width', 0)
    height = video_stream.get('height', 0)
    print(f"📹 Source: {width}x{height} @ {source_fps:.1f} fps")

# Extract audio
audio_path = WORK_DIR / "audio.wav"
print(f"🔊 Extracting audio...")
subprocess.run([
    "ffmpeg", "-y", "-i", str(source_video),
    "-vn", "-acodec", "pcm_s16le", str(audio_path)
], capture_output=True)

# Extract frames
frames = extract_frames(source_video, FRAMES_DIR)

# Deduplicate if enabled
if enable_deduplication:
    frames = deduplicate_frames(frames, dedup_threshold)

# Enhance frames
model = MODEL_MAP.get(scale_factor, "realesrgan-x4plus")
enhanced = enhance_frames(frames, ENHANCED_DIR, scale_factor, model)

# Reassemble
output_fps = target_fps if enable_interpolation else source_fps
output_file = WORK_DIR / f"restored.{output_format}"
reassemble_video(ENHANCED_DIR, output_file, output_fps, quality_crf, audio_path)

print("\n" + "="*60)
print("  ✅ RESTORATION COMPLETE!")
print("="*60)
print(f"Output: {output_file}")
print(f"Size: {output_file.stat().st_size / 1024 / 1024:.1f} MB")

## Step 4: Download Result

In [ ]:
# Download the restored video
from google.colab import files

output_file = Path("/content/framewright_output") / f"restored.{output_format}"
if output_file.exists():
    print(f"📥 Starting download: {output_file.name}")
    files.download(str(output_file))
else:
    print("❌ Output file not found. Run Step 3 first!")

## Optional: Save to Google Drive

In [ ]:
# Mount Google Drive and save
from google.colab import drive
drive.mount('/content/drive')

import shutil
output_file = Path("/content/framewright_output") / f"restored.{output_format}"
drive_path = Path("/content/drive/MyDrive/FrameWright_Output")
drive_path.mkdir(exist_ok=True)

dest = drive_path / output_file.name
shutil.copy(output_file, dest)
print(f"✅ Saved to Google Drive: {dest}")